In [1]:
%matplotlib widget

import sys  
sys.path.insert(0, '/home/cbisot/pycode/MscThesis/')
import pandas as pd
from amftrack.util import get_dates_datetime, get_dirname, get_data_info, update_plate_info, \
get_current_folders, get_folders_by_plate_id
import ast
from amftrack.plotutil import plot_t_tp1
from scipy import sparse
from datetime import datetime
from amftrack.pipeline.functions.node_id import orient
import pickle
import scipy.io as sio
from pymatreader import read_mat
from matplotlib import colors
import cv2
import imageio
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import frangi
from skimage import filters
from random import choice
import scipy.sparse
import os
from amftrack.pipeline.functions.extract_graph import from_sparse_to_graph, generate_nx_graph, sparse_to_doc
from skimage.feature import hessian_matrix_det
from amftrack.pipeline.functions.experiment_class_surf import Experiment
from amftrack.pipeline.paths.directory import run_parallel, find_state, find_state_extract, directory_scratch, directory_project, run_parallel_all_time


In [2]:
directory = directory_project
update_plate_info(directory)

In [3]:
update_plate_info(directory_scratch)

In [3]:
len(get_current_folders(directory))
all_folders = get_current_folders(directory)

In [29]:
plate = 22
plates = [22,10,76,26,152,102]

# folders = all_folders.loc[all_folders['Plate']==plate]
folders = all_folders.loc[all_folders['Plate'].isin(plates)]
print(len(folders))
# folders = all_folders.loc[all_folders['Plate']==plate*all_folders['/Analysis/skeleton_realigned_compressed.mat']]
folders = all_folders.loc[(all_folders['/Analysis/skeleton_realigned_compressed.mat'])&(all_folders['Plate'].isin(plates))]

print(len(folders))

351
336


In [30]:
folders

,folder,Plate,PrincePos,root,strain,medium,split,Temp,CrossDate,/Img/TileConfiguration.txt.registered,...,/Analysis/skeleton_masked_compressed.mat,/Analysis/skeleton_pruned_compressed.mat,/Analysis/transform.mat,/Analysis/transform_corrupt.mat,/Analysis/skeleton_realigned_compressed.mat,/Analysis/nx_graph_pruned.p,/Analysis/nx_graph_pruned_width.p,/Analysis/nx_graph_pruned_labeled.p,date,Pbait
14,20210110_1144_Plate38,76,38,Carrot,A5,001P100N,Y,NaN,20210108,True,...,True,True,True,False,True,True,False,False,"10.01.2021, 11:44:",-
21,20210112_1120_Plate29,22,29,Carrot,A5,001P100N,Y,NaN,20210107,True,...,True,True,True,False,True,True,False,False,"12.01.2021, 11:20:",-
28,20210111_1533_Plate36,10,36,Carrot,A5,001P100N,Y,NaN,20210107,True,...,True,True,True,False,True,True,False,False,"11.01.2021, 15:33:",-
39,20210110_0323_Plate30,26,39,Carrot,A5,001P100N,Y,NaN,20210104,True,...,True,True,True,False,True,True,False,False,"10.01.2021, 03:23:",-
48,20201229_0349_Plate40,102,40,Carrot,A5,001P100N,Y,25,20201226,True,...,True,True,True,False,True,True,False,False,"29.12.2020, 03:49:",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1516,20201231_1526_Plate32,152,32,Carrot,A5,001P100N,Y,NaN,20201224,True,...,True,True,True,False,True,True,False,False,"31.12.2020, 15:26:",-
1518,20201228_0343_Plate32,152,32,Carrot,A5,001P100N,Y,NaN,20201224,True,...,True,True,True,False,True,True,True,False,"28.12.2020, 03:43:",-
1520,20210108_1950_Plate40,102,40,Carrot,A5,001P100N,Y,25,20201226,True,...,True,True,True,False,True,True,False,False,"08.01.2021, 19:50:",NaN
1525,20201226_0738_Plate32,152,32,Carrot,A5,001P100N,Y,NaN,20201224,True,...,True,True,True,False,True,True,True,False,"26.12.2020, 07:38:",-


***Create graphs***

In [27]:
num_parallel = 100
time = '30:00'
args=[directory]
run_parallel('extract_nx_graph.py',args, folders, num_parallel, time,'extract_nx')

sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.


Submitted batch job 32569
Submitted batch job 32570
Submitted batch job 32571


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.


In [69]:
plates = [22,10,76,26,152,102]

folders = all_folders.loc[(all_folders['/Analysis/nx_graph_pruned.p'])&(all_folders['Plate'].isin(plates))]
len(folders)

336

***Extract Width***

In [70]:
num_parallel = 350
time = '00:10:00'
skip = True
args = [directory,skip]
run_parallel('extract_width.py',args,folders,num_parallel,time,'extract_width',cpus=128)

Submitted batch job 35690


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.


In [41]:
plates = [22,10,76,26,152,102]

folders = all_folders.loc[(1-all_folders['/Analysis/nx_graph_pruned_width.p'])&(all_folders['Plate'].isin(plates))&(all_folders['/Analysis/nx_graph_pruned.p'])]
# folders.loc[folders['folder'] == '20210110_1523_Plate30']
# int(list(folders['folder'])[0].split('_')[-1][5:])
len(folders)

0

In [4]:
plates = [22,10,76,26,152,102]

folders = all_folders.loc[(all_folders['/Analysis/nx_graph_pruned_width.p'])&(all_folders['Plate'].isin(plates))]
# folders.loc[folders['folder'] == '20210110_1523_Plate30']
# int(list(folders['folder'])[0].split('_')[-1][5:])
len(folders)

336

***Identify Nodes***

In [5]:
num_parallel = 6
time = '12:00:00'
args=[directory]
run_parallel_all_time('extract_nodes.py',args,folders,num_parallel,time,'node_id',cpus = 32)

Submitted batch job 42081
Submitted batch job 42082


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.


In [6]:
folders.columns

Index(['folder', 'Plate', 'PrincePos', 'root', 'strain', 'medium', 'split',
       'Temp', 'CrossDate', '/Img/TileConfiguration.txt.registered',
       '/Analysis/skeleton_compressed.mat',
       '/Analysis/skeleton_masked_compressed.mat',
       '/Analysis/skeleton_pruned_compressed.mat', '/Analysis/transform.mat',
       '/Analysis/transform_corrupt.mat',
       '/Analysis/skeleton_realigned_compressed.mat',
       '/Analysis/nx_graph_pruned.p', '/Analysis/nx_graph_pruned_width.p',
       '/Analysis/nx_graph_pruned_labeled.p', 'date', 'Pbait'],
      dtype='object')

In [7]:
plates = [22,10,76,26,152,102]

folders = all_folders.loc[(all_folders['/Analysis/nx_graph_pruned_labeled.p'])&(all_folders['Plate'].isin(plates))]
# folders.loc[folders['folder'] == '20210110_1523_Plate30']
# int(list(folders['folder'])[0].split('_')[-1][5:])
len(folders)

119

***Hyphae extraction***

In [13]:
num_parallel = 1
time = '5:00:00'
args = [plate, begin, end, directory]
run_parallel_all_time('hyphae_extraction.py',args,pd.DataFrame(),num_parallel,time,'hyphae')

In [16]:
 0.2 * 210 * 10**14 *30 *1.8

2.268e+17